In [18]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import joblib

In [19]:
queries_df = pd.read_csv('dataset_users_query.csv')
answers_df = pd.read_csv('answers.csv')

In [20]:
queries_df['query'] = queries_df['query'].str.lower().str.strip()
answers_df['query'] = answers_df['query'].str.lower().str.strip()

In [21]:
answers_df.head()

,Unnamed: 0,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,0,фарпост работа владивосток,NaN,NaN,NaN,NaN,общая фраза
1,1,кофейни вакансии,NaN,NaN,NaN,NaN,общая фраза
2,2,работа разнорабочие часовой,NaN,Рабочий,NaN,NaN,NaN
3,3,личный водитель на день,на неполный день,Водитель,NaN,NaN,NaN
4,4,работа от работодателя персональный водитель,NaN,Водитель,NaN,NaN,NaN


In [22]:
employment_df = answers_df.drop(['Unnamed: 0', 'по должности-лемме', 'по дополнительному признаку', 'по условиям', 'общие фразы'], axis = 1)
position_df = answers_df.drop(['Unnamed: 0', 'занятость', 'по дополнительному признаку', 'по условиям', 'общие фразы'], axis = 1)
additance_df = answers_df.drop(['Unnamed: 0', 'занятость', 'по должности-лемме', 'по условиям', 'общие фразы'], axis = 1)
conditions_df = answers_df.drop(['Unnamed: 0', 'занятость', 'по должности-лемме', 'по дополнительному признаку', 'общие фразы'], axis = 1)
phrase_df = answers_df.drop(['Unnamed: 0', 'занятость', 'по должности-лемме', 'по дополнительному признаку', 'по условиям'], axis = 1)

In [23]:
employment_keywords = list(answers_df['занятость'].unique()[1:])
position_keywords = list(answers_df['по должности-лемме'].unique()[1:])
additance_keywords = list(answers_df['по дополнительному признаку'].unique()[1:])
conditions_keywords = list(answers_df['по условиям'].unique()[1:])
common_phrases = ['работа', 'вакансии']

In [24]:
encoder = LabelEncoder()

employment_df['category_encoded'] = encoder.fit_transform(employment_df['занятость'])

In [25]:
employment_df

,query,занятость,category_encoded
0,фарпост работа владивосток,NaN,45
1,кофейни вакансии,NaN,45
2,работа разнорабочие часовой,NaN,45
3,личный водитель на день,на неполный день,36
4,работа от работодателя персональный водитель,NaN,45
...,...,...,...
14240,япония вакансии,NaN,45
14241,япония работа,NaN,45
14242,японский язык работа,NaN,45
14243,яппи вакансии,NaN,45


In [26]:
label_dict = {i: class_name for i, class_name in enumerate(encoder.classes_)}

In [27]:
label_dict[0] = 'Вахта'
label_dict[1] = 'Подработка'

In [28]:
vectorizer = TfidfVectorizer(max_features=5000)

X = vectorizer.fit_transform(employment_df['query'])
y = employment_df['category_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
model_query = LogisticRegression(max_iter=1000)
model_query.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [30]:
y_pred = model_query.predict(X_test)

In [31]:
# print(confusion_matrix(y_test, y_pred))

In [32]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           2       1.00      0.96      0.98        78
           3       0.67      0.67      0.67         3
           5       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         4
           9       0.00      0.00      0.00         2
          10       1.00      0.36      0.53        14
          11       0.00      0.00      0.00         1
          12       0.89      0.97      0.93        69
          13       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         3
          17       0.00      0.00      0.00         2
          18       0.00      0.00      0.00         2
          20       1.00      1.00      1.00         1
          24       0.00      0.00      0.00         1
          25       0.95      0.88      0.91        67
          28       0.00      0.00      0.00         1
          29       1.00    

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
# Сохранение модели
joblib.dump(model_query, 'employment_model.pkl')

['employment_model.pkl']

In [34]:
# employment_df['занятость'] = employment_df['занятость'].apply(lambda x: 0 if pd.isna(x) else 1)

In [36]:
joblib.dump(vectorizer, 'employment_vectorizer.pkl')

['employment_vectorizer.pkl']